In [2]:
num_tokens = 50_000_000 # numbrid pärit dictionary_learning_demo/demo_config.py-st
sae_batch_size = 2048
steps = int(num_tokens / sae_batch_size) # Total number of batches to train
log_steps = 100  # Log the training on wandb or print to console every log_steps

In [3]:
from nnsight import LanguageModel
from dictionary_learning import ActivationBuffer, AutoEncoder
from dictionary_learning.trainers import StandardTrainer
from dictionary_learning.training import trainSAE

device = "cuda:0"
model_name = "EleutherAI/pythia-70m-deduped" # can be any Huggingface model

model = LanguageModel(
    model_name,
    device_map=device,
)
submodule = model.gpt_neox.layers[1].mlp # layer 1 MLP
activation_dim = 512 # output dimension of the MLP
dictionary_size = 16 * activation_dim

# data must be an iterator that outputs strings
data = iter(
    [
        "This is some example data",
        "In real life, for training a dictionary",
        "you would need much more data than this",
    ]
)
buffer = ActivationBuffer(
    data=data,
    model=model,
    submodule=submodule,
    d_submodule=activation_dim, # output dimension of the model component
    n_ctxs=3e4,  # you can set this higher or lower dependong on your available memory
    device=device,
)  # buffer will yield batches of tensors of dimension = submodule's output dimension

trainer_cfg = {
    "trainer": StandardTrainer,
    "dict_class": AutoEncoder,
    "activation_dim": activation_dim,
    "dict_size": dictionary_size,
    "lr": 1e-3,
    "device": device,
}

# train the sparse autoencoder (SAE)
ae = trainSAE(
    data=buffer,  # you could also use another (i.e. pytorch dataloader) here instead of buffer
    trainer_configs=[trainer_cfg],
    steps=steps,
)

{'trainer': <class 'dictionary_learning.trainers.standard.StandardTrainer'>, 'dict_class': <class 'dictionary_learning.dictionary.AutoEncoder'>, 'activation_dim': 512, 'dict_size': 8192, 'lr': 0.001, 'device': 'cuda:0'}


TypeError: StandardTrainer.__init__() missing 3 required positional arguments: 'steps', 'layer', and 'lm_name'